In [1]:
import xlsxwriter as xl
import gurobipy
from gurobipy import *
import pandas as pd
import numpy as np
import openpyxl

ModuleNotFoundError: No module named 'xlsxwriter'

# Lecture des donnees pour extraire les csts

In [378]:
#small data
# path = "Data_OCP.xlsx"

#big data
path = "Big_Data_OCP.xlsx"

# # for test
# path = "Data_OCP.xlsx"

In [379]:
## Supply
supp_data = pd.read_excel(path,"Capacité source")

names = list(supp_data.columns)[1:]

rows = supp_data['MP'].values

dict_Supply = {}
for j in range (len(names)):
    l = supp_data[names[j]].values
    ocp = dict([(rows[i],l[i]) for i in range (len(rows)) ])
    dict_Supply[names[j]] = ocp

dict_Supply['OCP Jorf']['CAN']

0

In [380]:
## MP

mp_data = pd.read_excel(path,"MP")
#print(mp_data)

rows = mp_data['MP'].values
prices = mp_data['Prix'].values

dict_MP = dict([(rows[i],prices[i]) for i in range (len(prices))])



In [381]:
## Stockage

stock_data = pd.read_excel(path,"Stockage")
#print(stock_data)

Provinces = stock_data['Province '].values
cap_prov = stock_data['Capacité '].values

dict_Stockage = dict([(Provinces[i], cap_prov[i]) for i in range (len(Provinces))])



In [382]:
## Blender

blend_data = pd.read_excel(path,"Blenders")
#print(blend_data)

Provinces = blend_data['Zone'].values
blenders = blend_data['B'].values
capa_b = blend_data['CapacitéB'].values
capa_sb = blend_data['CapacitéSB'].values

dict_Blenders = {}

for i in range (len(blenders)):
    K = {}
    K["b"] = capa_b[i]
    K["sb"] = capa_sb[i]

    dict_Blenders[Provinces[i]] = K

#print (dict_Blenders)


In [383]:
## Demand


demand_data = pd.read_excel(path,"Demande").sort_values('Filière')
demand2_data = demand_data[["Q2","Q3","Q4","Q1"]]
recet_data = pd.read_excel(path,"Recettes").sort_values('Filière')
recet2_data = recet_data.iloc[:,2:]

#print(demand_data)

Provinces_dem = demand_data['Province'].values

filieres = demand_data['Filière'].values

dict_Demand = {}


for pro in Provinces:
    F = {}
    for i in range(len(filieres)):
        k={}
        if Provinces_dem[i]==pro:
            for col in demand2_data:
                k[col] = demand2_data[col].values[i]
            for col in recet2_data:
                k[col] = recet2_data[col].values[i]
            F [filieres[i]] = k



    dict_Demand[pro] = F




In [384]:
##Transport Amont


transAmont_data = pd.read_excel(path,"Cout de transport  Amont")

tranames = list(transAmont_data.columns)[1:]
trarows = transAmont_data['Zone'].values

dict_transAmont = {}
for j in range (len(tranames)):
    l = transAmont_data[tranames[j]].values
    ocp = dict([(trarows[i],l[i]) for i in range (len(trarows)) ])
    dict_transAmont[tranames[j]] = ocp


In [385]:
##Transport Aval

transAval_data = pd.read_excel(path,"Cout de transport aval")

tranames2 = list(transAval_data.columns)[1:]
trarows2 = transAval_data['Zone'].values
dict_transAval = {}
for j in range (len(tranames2)):
    l = transAval_data[tranames2[j]].values
    ocp = dict([(trarows2[i],l[i]) for i in range (len(trarows2)) ])
    dict_transAval[tranames2[j]] = ocp

In [386]:
# Options
donnees = pd.read_excel(path,"Options")
opt = donnees['Options'].values
rep = donnees['réponse'].values

Donnees = {}
for i in range (len(opt)):
    Donnees[opt[i]] = rep[i]

In [387]:
# périods
periods = ["Q2","Q3","Q4","Q1"]
periods 

['Q2', 'Q3', 'Q4', 'Q1']

In [388]:
MyDicts = [dict_Supply, dict_MP, dict_Stockage, dict_Blenders, dict_Demand, dict_transAmont, dict_transAval]

In [389]:
supply = [a for a in dict_Supply.keys()]
supply

['OCP Jorf',
 'OCP SAFI',
 'OCP Port Casablanca',
 'OCP Port Tanger',
 'OCP Port Nador']

In [390]:
Storage_MP = [a for a in dict_Stockage.keys()]
Province = [a for a in dict_Stockage.keys()]
Storage_MP

['Agadir Ida-Outanane',
 'Al Haouz',
 'Al Hoceima',
 'Aousserd',
 'Assa-zag',
 'Azilal',
 'Beni-Mellal',
 'Benslimane',
 'Berkane',
 'Berrechid',
 'Boujdour',
 'Boulemane',
 'Casablanca',
 'Chefchaouen',
 'Chichaoua',
 'Chtouka-Ait Baha',
 'Driouch',
 'El Hajeb',
 'El Jadida',
 'El Kelaades Sraghna',
 'Errachidia',
 'Essaouira',
 'EsSemara',
 'Fahs-Anjra',
 'Fes',
 'Figuig',
 'Fquih Ben Salah',
 'Guelmim',
 'Guercif',
 'Ifrane',
 'Inezgane-Ait Melloul',
 'Jrada',
 'Kenitra',
 'Khemisset',
 'Khenifra']

In [391]:
MP = [a for a in dict_MP.keys()]
MP


['CAN', 'DURAMON 26%', 'DAP', 'EM', 'MAP', 'MOP', 'TSP']

In [392]:
# Blenders = [a for a in dict_Blenders.keys() if dict_Blenders[a]['b'] !=0  ]
# SBlenders = [a for a in dict_Blenders.keys() if dict_Blenders[a]['sb'] !=0  ]
# print(Blenders)
# SBlenders

In [393]:
Storage_Fertilizer = Storage_MP.copy()

In [394]:
fertilizers = [ a+"_"+b for a in dict_Demand.keys() for b in dict_Demand[a].keys()]
filieres

array(['Agrumes', 'Agrumes', 'Agrumes', 'Agrumes', 'Agrumes', 'Agrumes',
       'Agrumes', 'Agrumes', 'Agrumes', 'Agrumes', 'Agrumes', 'Agrumes',
       'Agrumes', 'Agrumes', 'Amandiers', 'Amandiers', 'Amandiers',
       'Amandiers', 'Amandiers', 'Amandiers', 'Amandiers', 'Amandiers',
       'Amandiers', 'Amandiers', 'Amandiers', 'Amandiers', 'Amandiers',
       'Amandiers', 'Amandiers', 'Amandiers', 'Amandiers', 'Amandiers',
       'Amandiers', 'Amandiers', 'Amandiers', 'Amandiers', 'Amandiers',
       'Amandiers', 'Cereales', 'Cereales', 'Cereales', 'Cereales',
       'Cereales', 'Cereales', 'Cereales', 'Cereales', 'Cereales',
       'Cereales', 'Cereales', 'Cereales', 'Cereales', 'Cereales',
       'Cereales', 'Cereales', 'Cereales', 'Cereales', 'Cereales',
       'Cereales', 'Cereales', 'Cereales', 'Cereales', 'Cereales',
       'Cereales', 'Cereales', 'Cereales', 'Cereales', 'Fourrageres',
       'Fourrageres', 'Fourrageres', 'Fourrageres', 'Fourrageres',
       'Fourrageres', 'Fo

In [395]:
def pourcentage(e,i):
    L=e.split('_')
    filière = L.pop()
    if len(L)>1:
        province = L.join("_")
    else: province=L.pop()
    try:a=dict_Demand[province][filière][i]
    except:a="fertilizer not existing"
    return a
def DemandeE(e,t):
    L=e.split('_')
    filière = L.pop()
    if len(L)>1:
        province = L.join("_")
    else: province=L.pop()
    try:a=dict_Demand[province][filière][t]
    except:a="fertilizer not existing"
    return a
def fert(j):
    return [ j+"_"+b for b in dict_Demand[j].keys()]

In [396]:
# BIG M
M = Donnees["Cout_de_rupture"]

# Formulation 

In [397]:
model = Model("transportation_problem")
# Prob = pulp.LpProblem("transportation_problem",pulp.LpMinimize)

In [398]:
# Flow_Supply_StorageMP = pulp.LpVariable.dicts("Flow_Supply_StorageMP",(supply,Storage_MP,MP),lowBound=0,cat="Continuous")
Flow_Supply_StorageMP = model.addVars(supply,Storage_MP,MP,periods,vtype=GRB.CONTINUOUS,name="Flow_Supply_StorageMP")

In [399]:
# Flow_StorageMP_Blender = pulp.LpVariable.dicts("Flow_StorageMP_Blender",(Storage_MP,Blenders,MP),lowBound=0,cat="Continuous")
Flow_StorageMP_Blender = model.addVars(Storage_MP,Province,MP,periods,vtype=GRB.CONTINUOUS,name="Flow_StorageMP_Blender")

In [400]:
# Flow_StorageMP_SBlender =  pulp.LpVariable.dicts("Flow_StorageMP_SBlender",(Storage_MP,SBlenders,MP),lowBound=0,cat="Continuous")
Flow_StorageMP_SBlender = model.addVars(Storage_MP,Province,MP,periods,vtype=GRB.CONTINUOUS,name="Flow_StorageMP_SBlender")

In [401]:
# Flow_Supply_Blender = pulp.LpVariable.dicts("Flow_Supply_Blender",(supply,Blenders,MP),lowBound=0,cat="Continuous")
Flow_Supply_Blender = model.addVars(supply,Province,MP,periods,vtype=GRB.CONTINUOUS,name="Flow_Supply_Blender")


In [402]:
# Flow_Supply_SBlender = pulp.LpVariable.dicts("Flow_Supply_SBlender",(supply,SBlenders,MP),lowBound=0,cat="Continuous")
Flow_Supply_SBlender = model.addVars(supply,Province,MP,periods,vtype=GRB.CONTINUOUS,name="Flow_Supply_SBlender")

In [403]:
# Flow_Blender_StorageFertlizer = pulp.LpVariable.dicts("Flow_Blender_StorageFertlizer",(TBlenders,Storage_Fertilizer,fertilizers),lowBound=0,cat="Continuous")
Flow_Blender_StorageFertlizer = model.addVars(Province,Storage_Fertilizer,fertilizers,periods,vtype=GRB.CONTINUOUS,name="Flow_Blender_StorageFertlizer")

In [404]:
# Flow_StorageFertlizer_Province = pulp.LpVariable.dicts("Flow_StorageFertlizer_Province",(Storage_Fertilizer,fertilizers),lowBound=0,cat="Continuous")
Flow_StorageFertlizer_Province = model.addVars(Storage_Fertilizer,fertilizers,periods,vtype=GRB.CONTINUOUS,name="Flow_StorageFertlizer_Province")

In [405]:
# Flow_Blender_Province = pulp.LpVariable.dicts("Flow_Blender_Province",(Blenders,fertilizers),lowBound=0,cat="Continuous")
Flow_Blender_Province = model.addVars(Province,fertilizers,periods,vtype=GRB.CONTINUOUS,name="Flow_Blender_Province")

In [406]:
# Flow_SBlender_Province = pulp.LpVariable.dicts("Flow_SBlender_Province",(SBlenders,fertilizers),lowBound=0,cat="Continuous")
Flow_SBlender_Province = model.addVars(Province,fertilizers,periods,vtype=GRB.CONTINUOUS,name="Flow_SBlender_Province")

In [407]:
# Reliquat_Demand = pulp.LpVariable.dicts("Reliquat_Demand",(fertilizers),lowBound=0,cat="Continuous")
Reliquat_Demand = model.addVars(fertilizers,periods,vtype=GRB.CONTINUOUS,name="Reliquat_Demand")

In [408]:
# Stock_MP = pulp.LpVariable.dicts("Stock_MP",(Storage_MP,MP),lowBound=0,cat="Continuous")
Stock_MP = model.addVars(Storage_MP,MP,periods,vtype=GRB.CONTINUOUS,name="Stock_MP")
# fertilizers

In [409]:
# Stock_Fertlizer = pulp.LpVariable.dicts("Stock_Fertlizer",(Storage_Fertilizer,fertilizers),lowBound=0,cat="Continuous")
Stock_Fertlizer = model.addVars(Storage_Fertilizer,fertilizers,periods,vtype=GRB.CONTINUOUS,name="Stock_Fertlizer")
# Flow_Blender_StorageFertlizer

In [410]:
Storage_Invistement = model.addVars(Storage_MP,periods,vtype = GRB.BINARY,name = "Storage_Invistement")
Invistement_Cost = model.addVars(Storage_MP,vtype = GRB.BINARY,name = "Invistement_Cost")

In [411]:
Active_Blender = model.addVars(Storage_MP,periods,vtype=GRB.BINARY,name="Active_Blender")
Active_SBlender = model.addVars(Storage_MP,periods,vtype=GRB.BINARY,name="Active_SBlender")

In [412]:
Blender_Cost = model.addVars(Storage_MP,vtype=GRB.BINARY,name="Blender_Cost")
SBlender_Cost = model.addVars(Storage_MP,vtype=GRB.BINARY,name="SBlender_Cost")

In [413]:
Active_Downstream_Storage = model.addVars(Storage_MP,periods,vtype = GRB.BINARY,name = "Active_Downstream_Storage")

In [414]:
Active_Upstream_Storage = model.addVars(Storage_MP,periods,vtype = GRB.BINARY,name = "Active_Upstream_Storage")

# Activation

In [415]:
model.addConstr(quicksum(Active_Blender[(b,t)] + Active_SBlender[(b,t)] for b in Province for t in periods) <= Donnees["New blenders"] * M , "New blenders")

<gurobi.Constr *Awaiting Model Update*>

# Constraints Blenders Capacity


In [416]:
for t in periods:
    for b in Province:
        model.addConstr(quicksum(Flow_Blender_Province[(b,e,t)] for e in fertilizers) + quicksum(Flow_Blender_StorageFertlizer[(b,sf,e,t)] for sf in Storage_Fertilizer for e in fertilizers) <= dict_Blenders[b]['b'] +  Donnees["Capacité_blender_actiif"] * Active_Blender[(b,t)], "Capacité_blender_"+b+t )
        model.addConstr(quicksum(Flow_SBlender_Province[(b,e,t)] for e in fertilizers)  <= dict_Blenders[b]['sb'] + Donnees["Capacité_smart_blender_actiif"]*Active_SBlender[(b,t)]  , "Capacité_Sblender_"+b+t)

# Activate Blenders

In [417]:
for t in periods :
    for b in Province:
        model.addConstr(quicksum(Flow_Blender_Province[(b,e,t)] + Flow_SBlender_Province[(b,e,t)] for e in fertilizers) + quicksum(Flow_Blender_StorageFertlizer[(b,sf,e,t)] for sf in Storage_Fertilizer for e in fertilizers) <= dict_Blenders[b]['b'] + dict_Blenders[b]['sb'] + Donnees['Blender_min'] + M * (Active_Blender[(b,t)] + Active_SBlender[(b,t)]) , "activate_B_SB_"+b+"_"+t)
        model.addConstr(quicksum(Flow_Blender_Province[(b,e,t)] + Flow_SBlender_Province[(b,e,t)] for e in fertilizers) + quicksum(Flow_Blender_StorageFertlizer[(b,sf,e,t)] for sf in Storage_Fertilizer for e in fertilizers) >= dict_Blenders[b]['b'] + dict_Blenders[b]['sb'] + Donnees['Blender_min'] - M *(1-Active_Blender[(b,t)])  , "activate_B_SB_"+b+"_"+t)
        model.addConstr(quicksum(Flow_Blender_Province[(b,e,t)] + Flow_SBlender_Province[(b,e,t)] for e in fertilizers) + quicksum(Flow_Blender_StorageFertlizer[(b,sf,e,t)] for sf in Storage_Fertilizer for e in fertilizers) >= dict_Blenders[b]['b'] + dict_Blenders[b]['sb'] + Donnees['Blender_min'] - M *(1-Active_SBlender[(b,t)])  , "activate_B_SB_"+b+"_"+t)


# Cost Blender

In [418]:
for t in periods:
    for b in Province:
        model.addConstr(Blender_Cost[b] >= Active_Blender[(b,t)],"Cost_Blender_"+b+t)
        model.addConstr(SBlender_Cost[b] >= Active_SBlender[(b,t)],"Cost_SBlender_"+b+t)

# Constraints Supplies Capacity

In [419]:
# conservation ==
for t in periods:
    for s in supply:
        for m in MP:
            model.addConstr(quicksum(Flow_Supply_Blender[(s,b,m,t)] for b in Province) + quicksum(Flow_Supply_SBlender[(s,b,m,t)] for b in Province) + quicksum(Flow_Supply_StorageMP[(s,sm,m,t)] for sm in Storage_MP ) <= dict_Supply[s][m], "Capacité_supply_"+s+"_"+m + "_" +t)        

# Constraints Conservation of flow in Blenders

In [420]:
for t in periods:
    for b in Province :
         for m in MP:
            model.addConstr(quicksum(Flow_StorageMP_Blender[(sm,b,m,t)] for sm in Storage_MP ) + quicksum(Flow_Supply_Blender[(s,b,m,t)] for s in supply) - quicksum(pourcentage(e,m)*(Flow_Blender_Province[(b,e,t)] + Flow_Blender_StorageFertlizer[(b,sf,e,t)]) for sf in Storage_Fertilizer for e in fertilizers ) == 0, "Conservation_Blender_"+b+"_"+m+"_"+t)

In [421]:
for t in periods:
    for b in Province :
         for m in MP:
            model.addConstr(quicksum(Flow_StorageMP_SBlender[(sm,b,m,t)] for sm in Storage_MP ) + quicksum(Flow_Supply_SBlender[(s,b,m,t)] for s in supply) - quicksum(pourcentage(e,m)*(Flow_SBlender_Province[(b,e,t)]) for e in fertilizers ) == 0, "Conservation_SBlender_"+b+"_"+m + "_" +t)

# unsatisfied_demande

In [422]:
for t in periods:
    for e in fertilizers:
        model.addConstr(Reliquat_Demand[(e,t)] + quicksum(Flow_Blender_Province[(b,e,t)] for b in Province ) + quicksum(Flow_SBlender_Province[(b,e,t)] for b in Province ) + quicksum(Flow_StorageFertlizer_Province[(sf,e,t)] for sf in Storage_Fertilizer) - DemandeE(e,t) == 0,"unsatisfied_demande_"+e + "_" + t)

# Conservation_stockage

In [423]:
#For raw material

for sm in Storage_MP :
    for m in MP:
         model.addConstr(Stock_MP[(sm,m,"Q2")] - quicksum(Flow_Supply_StorageMP[(s,sm,m,"Q2")] for s in supply ) + quicksum(Flow_StorageMP_Blender[(sm,b,m,"Q2")] for b in Province ) + quicksum(Flow_StorageMP_SBlender[(sm,b,m,"Q2")] for b in Province) == 0 , "Conservation_stockage_MP_"+sm+"_"+m + "_" + "Q2" )
    

In [424]:
#For raw material
for t in periods[1:]:
    for sm in Storage_MP :
        for m in MP:
             model.addConstr(Stock_MP[(sm,m,t)] - Stock_MP[(sm,m,periods[periods.index(t)-1])] - quicksum(Flow_Supply_StorageMP[(s,sm,m,t)] for s in supply ) + quicksum(Flow_StorageMP_Blender[(sm,b,m,t)] for b in Province ) + quicksum(Flow_StorageMP_SBlender[(sm,b,m,t)] for b in Province) == 0 , "Conservation_stockage_MP_"+sm+"_"+m )           

In [425]:
#For Fertilizers
for sf in Storage_Fertilizer :
    for e in fertilizers :
        model.addConstr(Stock_Fertlizer[(sf,e,"Q2")] - quicksum(Flow_Blender_StorageFertlizer[(b,sf,e,"Q2")] for b in Province) + Flow_StorageFertlizer_Province[(sf,e,"Q2")] == 0 ,"Conservation_Storage_Fertilizer_"+sf+"_"+e +"_Q2")

In [426]:
#For Fertilizers
for t in periods[1:]:
    for sf in Storage_Fertilizer :
        for e in fertilizers :
            model.addConstr(Stock_Fertlizer[(sf,e,t)] - Stock_Fertlizer[(sf,e,periods[periods.index(t)-1])] - quicksum(Flow_Blender_StorageFertlizer[(b,sf,e,t)] for b in Province) + Flow_StorageFertlizer_Province[(sf,e,t)] == 0 ,"Conservation_Storage_Fertilizer_"+sf+"_"+e + "_"+t)

# Contraints Capacity storage

In [427]:
Capacite_stock_actif = Donnees["Capacité_stock_actif"]
for t in periods: 
    for p in Storage_MP:
        model.addConstr(quicksum(Stock_MP[(p,m,t)] for m in MP) + quicksum(Stock_Fertlizer[(p,e,t)] for e in fertilizers)  <= dict_Stockage[p] + Capacite_stock_actif*Storage_Invistement[(p,t)] , "Constraint_Capacité_Storage_"+p + "_"+t) 

# Possibility of activation storage 

In [428]:
New_Storages =  Donnees["New stockages"]
model.addConstr(quicksum(Active_Upstream_Storage[(p,t)]+Active_Downstream_Storage[(p,t)] for p in Storage_MP for t in periods) <= M*New_Storages ,"Possibility of storage activation")

<gurobi.Constr *Awaiting Model Update*>

# Constraint of activation storage

In [429]:
Stockage_min =  Donnees["Stockage_min"]

for p in Storage_MP:
    for t in periods :
        model.addConstr(Active_Downstream_Storage[(p,t)] + Active_Upstream_Storage[(p,t)] <= 1)
        model.addConstr(quicksum(Stock_MP[(p,m,t)] for m in MP) + quicksum(Stock_Fertlizer[(p,e,t)] for e in fertilizers)  <= dict_Stockage[p] + Stockage_min + M*(Active_Downstream_Storage[(p,t)] + Active_Upstream_Storage[(p,t)]) , "Constraint_Activation_Storage_"+p +"_"+ t)
        model.addConstr(quicksum(Stock_MP[(p,m,t)] for m in MP) + quicksum(Stock_Fertlizer[(p,e,t)] for e in fertilizers)  >= dict_Stockage[p] + Stockage_min - M*(1-(Active_Downstream_Storage[(p,t)] + Active_Upstream_Storage[(p,t)])) , "Constraint_Activation_Storage_"+p +"_"+ t)
        model.addConstr(Invistement_Cost[p] >= Active_Downstream_Storage[(p,t)] + Active_Upstream_Storage[(p,t)],"Constraint_Invistement_Cost" + p + "_" +t)

# The moment of truth Objective function

In [430]:
Cout_transport_amont = quicksum( quicksum(dict_transAmont[s][p]*(Flow_Supply_Blender[(s,p,m,t)]) for s in supply for m in MP for p in Province ) + quicksum(dict_transAmont[s][p]*(Flow_Supply_SBlender[(s,p,m,t)]) for s in supply for m in MP for p in Province ) + quicksum(dict_transAmont[s][p]*Flow_Supply_StorageMP[(s,p,m,t)] for s in supply for p in Storage_MP for m in MP ) for t in periods)

In [431]:
# on deux province on les appelle i et j 
Cout_transport_aval = quicksum( quicksum(dict_transAval[b][j] * Flow_Blender_StorageFertlizer[(b,j,e,t)] for b in Province for j in Storage_Fertilizer for e in fertilizers) + quicksum(dict_transAval[i][j]*(Flow_StorageFertlizer_Province[(i,ej,t)]) for i in Storage_MP for j in Storage_MP for e in fertilizers for ej in fert(j)  ) + quicksum(dict_transAval[i][j]*(Flow_Blender_Province[(i,ej,t)]) for i in Province for j in Storage_MP for ej in fert(j)) + quicksum(dict_transAval[i][j]*(Flow_SBlender_Province[(i,ej,t)]) for i in Province for j in Storage_MP for ej in fert(j)) + quicksum(dict_transAval[i][j]*(Flow_StorageMP_Blender[(i,j,m,t)])for i in Storage_MP for j in Province for m in MP) +quicksum(dict_transAval[i][j]*(Flow_StorageMP_SBlender[(i,j,m,t)])for i in Storage_MP for j in Province for m in MP) for t in periods)  

In [432]:
Cout_stockage_unit = 10
Cout_stockage =quicksum( quicksum((Cout_stockage_unit*(quicksum(Stock_MP[(S,m,t)] for m in MP)+quicksum(Stock_Fertlizer[(S,e,t)] for e in fertilizers))) for S in Storage_MP) for t in periods) 

In [433]:
Cout_blending_unit = Donnees["Cout_blending"]
Cout_blending = quicksum( quicksum((Cout_blending_unit*(quicksum(Flow_Supply_Blender[(s,b,m,t)] for m in MP for s in supply)+quicksum(Flow_StorageMP_Blender[(sm,b,m,t)] for sm in Storage_MP for m in MP))) for b in Province) for t in periods) 

In [434]:
Cout_Sblending_unit = Donnees["Cout_smart_blending"]
Cout_Sblending = quicksum( quicksum((Cout_Sblending_unit*(quicksum(Flow_Supply_SBlender[(s,b,m,t)] for m in MP for s in supply)+quicksum(Flow_StorageMP_SBlender[(sm,b,m,t)] for sm in Storage_MP for m in MP))) for b in Province) for t in periods)  

In [435]:
Investissement_B = quicksum(Blender_Cost[b] * Donnees["Cout_blender"] + SBlender_Cost[b] * Donnees["Cout_sblender"] for b in Province)

In [436]:
Cout_Stockage_building = Donnees["Cout_de_stockage_MP"]
Cout_Invistement_Stockage =  quicksum(Invistement_Cost[p]*Cout_Stockage_building for p in Storage_MP) 

In [437]:
model.setObjective(Cout_transport_amont + Cout_transport_aval + Cout_stockage + Cout_blending + Cout_Sblending + Investissement_B + Cout_Invistement_Stockage + M * quicksum(Reliquat_Demand[(e,t)] for e in fertilizers for t in periods),GRB.MINIMIZE)

In [438]:
model.update()
print(model)

<gurobi.Model MIP instance transportation_problem: 44218 constrs, 1582221 vars, No parameter changes>


In [439]:
model.write('test.lp')
# model.Params.MIPGap = 0.02
model.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 44218 rows, 1582221 columns and 17445761 nonzeros
Model fingerprint: 0x9b69d56b
Variable types: 1581416 continuous, 805 integer (805 binary)
Coefficient statistics:
  Matrix range     [1e-03, 1e+05]
  Objective range  [1e+01, 1e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 5e+08]
Found heuristic solution: objective 5.082927e+10
Presolve removed 91 rows and 9626 columns (presolve time = 5s) ...
Presolve removed 460 rows and 9732 columns (presolve time = 10s) ...
Presolve removed 598 rows and 16031 columns (presolve time = 15s) ...
Presolve removed 598 rows and 16066 columns (presolve time = 20s) ...
Presolve removed 619 rows and 17161 columns (presolve time = 26s) ...
Presolve removed 619 rows and 17161 columns (presolve time = 31s) ...
Presolve removed 619 rows and 17161 columns (presolve time = 50s) ...
Presolve re

In [440]:
print("objective function value is : %f" %model.objVal)
model.printAttr(["X"])#,"Obj","SAObjLow","SAObjUp"])
# model.printAttr(["X","RC","LB","SALBUp","SALBLow","UB","SAUBUp","SAUBLow"])
# model.printAttr(["Sense","Slack","Pi","RHS","SARHSUp","SARHSLow"])

objective function value is : 2004708381.995955

    Variable            X 
-------------------------
Flow_Supply_StorageMP[OCP Jorf,Berrechid,DAP,Q4]      1106.07 
Flow_Supply_StorageMP[OCP Jorf,Berrechid,DAP,Q1]      922.788 
Flow_Supply_StorageMP[OCP Jorf,Berrechid,EM,Q4]      748.227 
Flow_Supply_StorageMP[OCP Jorf,Berrechid,EM,Q1]      624.239 
Flow_Supply_StorageMP[OCP Jorf,Berrechid,MAP,Q4]      101.296 
Flow_Supply_StorageMP[OCP Jorf,Berrechid,MAP,Q1]      124.018 
Flow_Supply_StorageMP[OCP Jorf,Boujdour,DAP,Q4]      762.433 
Flow_Supply_StorageMP[OCP Jorf,Boujdour,DAP,Q1]      563.392 
Flow_Supply_StorageMP[OCP Jorf,Boujdour,EM,Q4]      515.764 
Flow_Supply_StorageMP[OCP Jorf,Boujdour,EM,Q1]      381.118 
Flow_Supply_StorageMP[OCP Jorf,Boujdour,MAP,Q4]      128.044 
Flow_Supply_StorageMP[OCP Jorf,Boujdour,MAP,Q1]      94.0917 
Flow_Supply_StorageMP[OCP Jorf,El Hajeb,DAP,Q2]       1494.1 
Flow_Supply_StorageMP[OCP Jorf,El Hajeb,DAP,Q3]      1490.41 
Flow_Supply_StorageMP[OCP Jo

Flow_StorageMP_Blender[El Jadida,Casablanca,MAP,Q3]      1813.07 
Flow_StorageMP_Blender[El Jadida,Casablanca,MAP,Q4]      2327.51 
Flow_StorageMP_Blender[El Jadida,Casablanca,MAP,Q1]      1666.03 
Flow_StorageMP_Blender[Essaouira,Agadir Ida-Outanane,TSP,Q4]      47.7942 
Flow_StorageMP_SBlender[Berrechid,Boujdour,CAN,Q4]      289.345 
Flow_StorageMP_SBlender[Berrechid,Boujdour,CAN,Q1]       350.06 
Flow_StorageMP_SBlender[Berrechid,Boujdour,DURAMON 26,Q4]      325.513 
Flow_StorageMP_SBlender[Berrechid,Boujdour,DURAMON 26,Q1]      393.817 
Flow_StorageMP_SBlender[Berrechid,Boujdour,DAP,Q4]      1106.07 
Flow_StorageMP_SBlender[Berrechid,Boujdour,DAP,Q1]      922.788 
Flow_StorageMP_SBlender[Berrechid,Boujdour,EM,Q4]      748.227 
Flow_StorageMP_SBlender[Berrechid,Boujdour,EM,Q1]      624.239 
Flow_StorageMP_SBlender[Berrechid,Boujdour,MAP,Q4]      101.296 
Flow_StorageMP_SBlender[Berrechid,Boujdour,MAP,Q1]      124.018 
Flow_StorageMP_SBlender[Berrechid,Boujdour,MOP,Q4]      344.405 


Flow_StorageMP_SBlender[El Hajeb,Errachidia,DAP,Q4]      819.457 
Flow_StorageMP_SBlender[El Hajeb,Errachidia,DAP,Q1]      1001.04 
Flow_StorageMP_SBlender[El Hajeb,Errachidia,EM,Q4]      554.339 
Flow_StorageMP_SBlender[El Hajeb,Errachidia,EM,Q1]      677.173 
Flow_StorageMP_SBlender[El Hajeb,Errachidia,MAP,Q4]       127.15 
Flow_StorageMP_SBlender[El Hajeb,Errachidia,MAP,Q1]      115.423 
Flow_StorageMP_SBlender[El Hajeb,Errachidia,MOP,Q4]       432.31 
Flow_StorageMP_SBlender[El Hajeb,Errachidia,MOP,Q1]      392.437 
Flow_StorageMP_SBlender[El Hajeb,Errachidia,TSP,Q4]      236.776 
Flow_StorageMP_SBlender[El Hajeb,Errachidia,TSP,Q1]      187.749 
Flow_StorageMP_SBlender[El Hajeb,Fes,CAN,Q2]      103.965 
Flow_StorageMP_SBlender[El Hajeb,Fes,CAN,Q3]      103.963 
Flow_StorageMP_SBlender[El Hajeb,Fes,CAN,Q4]      324.149 
Flow_StorageMP_SBlender[El Hajeb,Fes,CAN,Q1]      324.149 
Flow_StorageMP_SBlender[El Hajeb,Fes,DURAMON 26,Q2]       116.96 
Flow_StorageMP_SBlender[El Hajeb,Fes,DUR

Flow_StorageMP_SBlender[El Jadida,Beni-Mellal,MAP,Q1]      181.911 
Flow_StorageMP_SBlender[El Jadida,Benslimane,DAP,Q2]      117.918 
Flow_StorageMP_SBlender[El Jadida,Benslimane,DAP,Q3]       21.021 
Flow_StorageMP_SBlender[El Jadida,Benslimane,DAP,Q4]      1334.04 
Flow_StorageMP_SBlender[El Jadida,Benslimane,DAP,Q1]      666.162 
Flow_StorageMP_SBlender[El Jadida,Benslimane,EM,Q2]       79.768 
Flow_StorageMP_SBlender[El Jadida,Benslimane,EM,Q3]      14.2201 
Flow_StorageMP_SBlender[El Jadida,Benslimane,EM,Q4]      902.439 
Flow_StorageMP_SBlender[El Jadida,Benslimane,EM,Q1]      450.639 
Flow_StorageMP_SBlender[El Jadida,Benslimane,MAP,Q2]      21.0067 
Flow_StorageMP_SBlender[El Jadida,Benslimane,MAP,Q3]      2.46145 
Flow_StorageMP_SBlender[El Jadida,Benslimane,MAP,Q4]      170.041 
Flow_StorageMP_SBlender[El Jadida,Benslimane,MAP,Q1]      113.091 
Flow_StorageMP_SBlender[El Jadida,Berrechid,DAP,Q2]      583.036 
Flow_StorageMP_SBlender[El Jadida,Berrechid,DAP,Q3]      740.652 


Flow_StorageMP_SBlender[El Jadida,Khenifra,DAP,Q1]      663.299 
Flow_StorageMP_SBlender[El Jadida,Khenifra,EM,Q2]      59.3256 
Flow_StorageMP_SBlender[El Jadida,Khenifra,EM,Q4]      533.655 
Flow_StorageMP_SBlender[El Jadida,Khenifra,EM,Q1]      448.702 
Flow_StorageMP_SBlender[El Jadida,Khenifra,MAP,Q2]      23.7719 
Flow_StorageMP_SBlender[El Jadida,Khenifra,MAP,Q4]      217.099 
Flow_StorageMP_SBlender[El Jadida,Khenifra,MAP,Q1]      197.254 
Flow_StorageMP_SBlender[El Kelaades Sraghna,Azilal,TSP,Q4]      363.412 
Flow_StorageMP_SBlender[El Kelaades Sraghna,Azilal,TSP,Q1]       296.34 
Flow_StorageMP_SBlender[Essaouira,Agadir Ida-Outanane,TSP,Q4]       157.37 
Flow_StorageMP_SBlender[Essaouira,Agadir Ida-Outanane,TSP,Q1]       106.42 
Flow_StorageMP_SBlender[Essaouira,Assa-zag,TSP,Q4]      85.1402 
Flow_StorageMP_SBlender[Essaouira,Assa-zag,TSP,Q1]      83.8101 
Flow_StorageMP_SBlender[Essaouira,Chtouka-Ait Baha,TSP,Q4]      142.854 
Flow_StorageMP_SBlender[Essaouira,Chtouka-Ait B

...

Flow_SBlender_Province[Fquih Ben Salah,Beni-Mellal_Legumineuses,Q3]       222.69 
Flow_SBlender_Province[Fquih Ben Salah,Beni-Mellal_Legumineuses,Q4]      1539.39 
Flow_SBlender_Province[Fquih Ben Salah,Beni-Mellal_Maraichage,Q2]      59.1983 
Flow_SBlender_Province[Fquih Ben Salah,Beni-Mellal_Maraichage,Q3]         59.2 
Flow_SBlender_Province[Fquih Ben Salah,Beni-Mellal_Maraichage,Q4]       710.38 
Flow_SBlender_Province[Fquih Ben Salah,Beni-Mellal_Oleagineuses,Q2]    0.0326405 
Flow_SBlender_Province[Fquih Ben Salah,Beni-Mellal_Oleagineuses,Q3]         0.03 
Flow_SBlender_Province[Fquih Ben Salah,Beni-Mellal_Oliviers,Q2]       101.58 
Flow_SBlender_Province[Fquih Ben Salah,Beni-Mellal_Oliviers,Q3]       101.58 
Flow_SBlender_Province[Fquih Ben Salah,Beni-Mellal_Rosacees,Q2]      3.92541 
Flow_SBlender_Province[Fquih Ben Salah,Beni-Mellal_Rosacees,Q3]         3.93 
Flow_SBlender_Province[Fquih Ben Salah,Beni-Mellal_Rosacees,Q4]         47.1 
Flow_SBlender_Province[Fquih Ben Salah,Ben

Flow_SBlender_Province[Jrada,Figuig_Cereales,Q3]       119.85 
Flow_SBlender_Province[Jrada,Figuig_Fourrageres,Q2]      39.2663 
Flow_SBlender_Province[Jrada,Figuig_Fourrageres,Q3]        39.27 
Flow_SBlender_Province[Jrada,Figuig_Legumineuses,Q2]      27.4673 
Flow_SBlender_Province[Jrada,Figuig_Legumineuses,Q3]        27.47 
Flow_SBlender_Province[Jrada,Figuig_Maraichage,Q2]      7.35089 
Flow_SBlender_Province[Jrada,Figuig_Maraichage,Q3]         7.35 
Flow_SBlender_Province[Jrada,Figuig_Oliviers,Q2]       47.147 
Flow_SBlender_Province[Jrada,Figuig_Oliviers,Q3]        47.15 
Flow_SBlender_Province[Jrada,Figuig_Oliviers,Q4]       309.54 
Flow_SBlender_Province[Jrada,Figuig_Palmiers-Dattiers,Q2]      8.38169 
Flow_SBlender_Province[Jrada,Figuig_Palmiers-Dattiers,Q3]         8.38 
Flow_SBlender_Province[Jrada,Figuig_Rosacees,Q2]      0.11479 
Flow_SBlender_Province[Jrada,Figuig_Rosacees,Q3]         0.11 
Flow_SBlender_Province[Jrada,Guercif_Fourrageres,Q4]      1624.56 
Flow_SBlender_P

Flow_SBlender_Province[Khenifra,Khenifra_Vignes,Q2]     0.296323 
Flow_SBlender_Province[Khenifra,Khenifra_Vignes,Q1]      1.77794 
Stock_Fertlizer[Agadir Ida-Outanane,Agadir Ida-Outanane_Oliviers,Q3]         2592 
Stock_Fertlizer[Agadir Ida-Outanane,Agadir Ida-Outanane_Rosacees,Q3]        83.35 
Stock_Fertlizer[Al Hoceima,Al Hoceima_Amandiers,Q3]       718.63 
Stock_Fertlizer[Al Hoceima,Al Hoceima_Fourrageres,Q3]       511.91 
Stock_Fertlizer[Al Hoceima,Al Hoceima_Industrielles,Q3]        14.69 
Stock_Fertlizer[Al Hoceima,Al Hoceima_Oliviers,Q3]       108.46 
Stock_Fertlizer[Al Hoceima,Al Hoceima_Rosacees,Q3]        18.42 
Stock_Fertlizer[Al Hoceima,Al Hoceima_Vignes,Q3]       811.62 
Stock_Fertlizer[Benslimane,Benslimane_Cereales,Q3]        152.6 
Stock_Fertlizer[Benslimane,Benslimane_Fourrageres,Q3]       340.75 
Stock_Fertlizer[Benslimane,Benslimane_Legumineuses,Q3]       116.45 
Stock_Fertlizer[Benslimane,Benslimane_Maraichage,Q3]       114.82 
Stock_Fertlizer[Benslimane,Bensliman

Storage_Invistement[Boulemane,Q4]            1 
Storage_Invistement[Boulemane,Q1]            1 
Storage_Invistement[Casablanca,Q2]            1 
Storage_Invistement[Casablanca,Q3]            1 
Storage_Invistement[Casablanca,Q4]            1 
Storage_Invistement[Casablanca,Q1]            1 
Storage_Invistement[Chefchaouen,Q2]            1 
Storage_Invistement[Chefchaouen,Q3]            1 
Storage_Invistement[Chefchaouen,Q4]            1 
Storage_Invistement[Chefchaouen,Q1]            1 
Storage_Invistement[Chichaoua,Q2]            1 
Storage_Invistement[Chichaoua,Q3]            1 
Storage_Invistement[Chichaoua,Q4]            1 
Storage_Invistement[Chichaoua,Q1]            1 
Storage_Invistement[Chtouka-Ait Baha,Q2]            1 
Storage_Invistement[Chtouka-Ait Baha,Q3]            1 
Storage_Invistement[Chtouka-Ait Baha,Q4]            1 
Storage_Invistement[Chtouka-Ait Baha,Q1]            1 
Storage_Invistement[Driouch,Q2]            1 
Storage_Invistement[Driouch,Q3]            1 
Stor

Active_SBlender[Fquih Ben Salah,Q4]            1 
Active_SBlender[Fquih Ben Salah,Q1]            1 
Active_SBlender[Guelmim,Q4]            1 
Active_SBlender[Guelmim,Q1]            1 
Active_SBlender[Guercif,Q2]            1 
Active_SBlender[Guercif,Q3]            1 
Active_SBlender[Guercif,Q4]            1 
Active_SBlender[Guercif,Q1]            1 
Active_SBlender[Ifrane,Q2]            1 
Active_SBlender[Ifrane,Q3]            1 
Active_SBlender[Ifrane,Q4]            1 
Active_SBlender[Ifrane,Q1]            1 
Active_SBlender[Inezgane-Ait Melloul,Q4]            1 
Active_SBlender[Inezgane-Ait Melloul,Q1]            1 
Active_SBlender[Jrada,Q2]            1 
Active_SBlender[Jrada,Q3]            1 
Active_SBlender[Jrada,Q4]            1 
Active_SBlender[Jrada,Q1]            1 
Active_SBlender[Kenitra,Q2]            1 
Active_SBlender[Kenitra,Q3]            1 
Active_SBlender[Kenitra,Q4]            1 
Active_SBlender[Kenitra,Q1]            1 
Active_SBlender[Khemisset,Q4]            1 
Ac

In [441]:
Flow_Blender_Province.keys()[0][2]

'Q2'

In [473]:
marge_distributeur = Donnees['Marge distributeur']
marge_revendeur = Donnees['Marge revendeur']


In [474]:
pathr = "222Results_small_data.xlsx"
outWorkBook = xl.Workbook(pathr)

def Write_flow_blender_province(outWorkBook):
    outSheet = outWorkBook.add_worksheet("flow_blender_province")
    columns = ["Blender","Province","Culture","Periode","Type of Blender","Quantity",'freight_cost',"total_transport_cost","blending_cost"]
    for i in range(len(columns)):
        outSheet.write(0,i,columns[i])
    i = 1
    for blender in Province :
        for fertilizer in fertilizers:
            for periode in periods :
                Province_Culture = fertilizer.split("_")
                
                #B
                B = Flow_Blender_Province[blender,fertilizer,periode].X
                if B != 0:
                    outSheet.write(i,0,blender)
                    outSheet.write(i,1,Province_Culture[0])
                    outSheet.write(i,2,Province_Culture[1])
                    outSheet.write(i,3,periode)
                    outSheet.write(i,4,"B")
                    outSheet.write(i,5,B)
                    outSheet.write(i,6,dict_transAval[blender][Province_Culture[0]])
                    outSheet.write(i,7,dict_transAval[blender][Province_Culture[0]]*B)
                    outSheet.write(i,8,Donnees["Cout_blending"]*B)
                    i+=1
                
                #SB
                SB = Flow_SBlender_Province[blender,fertilizer,periode].X
                if SB != 0:
                    outSheet.write(i,0,blender)
                    outSheet.write(i,1,Province_Culture[0])
                    outSheet.write(i,2,Province_Culture[1])
                    outSheet.write(i,3,periode)
                    outSheet.write(i,4,"SB")
                    outSheet.write(i,5,SB)
                    outSheet.write(i,6,dict_transAval[blender][Province_Culture[0]])
                    outSheet.write(i,7,dict_transAval[blender][Province_Culture[0]]*SB)
                    outSheet.write(i,8,Donnees["Cout_smart_blending"]*SB)
                    i+=1
    
def Write_cost_to_blender(outWorkBook):
    outSheet = outWorkBook.add_worksheet("cost_to_blender")
    columns = ["Blender","MP","Periode","Type of Blender","volume_supply","cost_supply","volume_st","cost_st","average_cost_b"]
    for i in range(len(columns)):
        outSheet.write(0,i,columns[i])
    
    i = 1
    for blender in Province :
        for mp in MP:
            for periode in periods :
                #B
                volume_supply = 0
                cout_supply = 0
                
                volume_st = 0
                cout_st = 0
                
                for s in supply:
                    
                    s_blender = Flow_Supply_Blender[s,blender,mp,periode].X
                    volume_supply += s_blender
                    cout_supply += s_blender*(dict_transAmont[s][blender]+dict_MP[mp])
                    
                for st in Storage_MP:
                    st_blender = Flow_StorageMP_Blender[st,blender,mp,periode].X
                    cout_transport_amont = 0
                    # code for calculating transportation cost from supply to st
#                     if st_blender != 0 :
#                         volume_s_st = 0
#                         cost_s_st = 0 
#                         for s in supply:
#                             s_st = Flow_Supply_StorageMP[s,st,mp,periode].X
#                             volume_s_st +=s_st
#                             cost_s_st +=s_st*dict_transAmont[s][st]
#                         if cost_s_st != 0:
#                             cout_transport_amont = cost_s_st/cost_s_st
                    
                    volume_st += st_blender
                    cout_st += st_blender*(Donnees["Cout_de_stockage_MP"] + dict_transAval[st][blender]+dict_MP[mp])
                
                if volume_supply != 0 or volume_st != 0:
                    outSheet.write(i,0,blender)
                    outSheet.write(i,1,mp)
                    outSheet.write(i,2,periode)
                    outSheet.write(i,3,"B")
                    outSheet.write(i,4,volume_supply)
                    outSheet.write(i,5,cout_supply)
                    outSheet.write(i,6,volume_st)
                    outSheet.write(i,7,cout_st)
                    outSheet.write(i,8,(cout_st+cout_supply)/(volume_supply+volume_st))
                    i+=1
                
                #SB
                volume_supply = 0
                cout_supply = 0
                
                volume_st = 0
                cout_st = 0
                
                for s in supply:
                    s_blender = Flow_Supply_SBlender[s,blender,mp,periode].X
                    volume_supply += s_blender
                    cout_supply += s_blender*(dict_transAmont[s][blender]+dict_MP[mp])
                
                for st in Storage_MP:
                    st_blender = Flow_StorageMP_SBlender[st,blender,mp,periode].X
                    volume_st += st_blender
                    cout_transport_amont = 0
                    # code for calculating transportation cost from supply to st
#                     if st_blender != 0 :
#                         volume_s_st = 0
#                         cost_s_st = 0 
#                         for s in supply:
#                             s_st = Flow_Supply_StorageMP[s,st,mp,periode].X
#                             volume_s_st +=s_st
#                             cost_s_st +=s_st*dict_transAmont[s][st]
#                         if cost_s_st != 0:
#                             cout_transport_amont = cost_s_st/cost_s_st
                    
                    cout_st += st_blender*(cout_transport_amont + Donnees["Cout_de_stockage_MP"] + dict_transAval[st][blender]+dict_MP[mp])
                 
                if volume_supply != 0 or volume_st != 0:
                    outSheet.write(i,0,blender)
                    outSheet.write(i,1,mp)
                    outSheet.write(i,2,periode)
                    outSheet.write(i,3,"SB")
                    outSheet.write(i,4,volume_supply)
                    outSheet.write(i,5,cout_supply)
                    outSheet.write(i,6,volume_st)
                    outSheet.write(i,7,cout_st)
                    outSheet.write(i,8,(cout_st+cout_supply)/(volume_supply+volume_st))
                    i+=1

In [475]:
print(pd.__version__)

0.23.4


In [476]:


def Write_flow_stockage_mp_blender(outWorkBook):
    outSheet = outWorkBook.add_worksheet("flow_stockage_mp_blender")
    columns = ["Stockage_MP","Blender","MP","Periode","Type of Blender","Quantité",'freight_cost',"total_cost","averege_cost"]
    for i in range(len(columns)):
        outSheet.write(0,i,columns[i])
    i = 1
    Storage_MP,Province,MP,periods
    for st in Storage_MP :
        for blender in Province:
            for mp in MP :
                for periode in periods:
                    #B
                    B = Flow_StorageMP_Blender[st,blender,mp,periode].X
                    if B != 0:
                        outSheet.write(i,0,st)
                        outSheet.write(i,1,blender)
                        outSheet.write(i,2,mp)
                        outSheet.write(i,3,periode)
                        outSheet.write(i,4,"B")
                        outSheet.write(i,5,B)
                        outSheet.write(i,6,dict_transAval[st][blender])
                        outSheet.write(i,7,(Donnees["Cout_de_stockage_MP"] +dict_transAval[st][blender])*B)
                        outSheet.write(i,8,Donnees["Cout_de_stockage_MP"] +dict_transAval[st][blender])
                        i+=1

                    #SB
                    SB = Flow_StorageMP_SBlender[st,blender,mp,periode].X
                    if SB != 0:
                        outSheet.write(i,0,st)
                        outSheet.write(i,1,blender)
                        outSheet.write(i,2,mp)
                        outSheet.write(i,3,periode)
                        outSheet.write(i,4,"SB")
                        outSheet.write(i,5,SB)
                        outSheet.write(i,6,dict_transAval[st][blender])
                        outSheet.write(i,7,(Donnees["Cout_de_stockage_MP"] +dict_transAval[st][blender])*SB)
                        outSheet.write(i,8,Donnees["Cout_de_stockage_MP"] +dict_transAval[st][blender])
                        i+=1

In [477]:
Write_flow_stockage_mp_blender(outWorkBook)
Write_flow_blender_province(outWorkBook)
Write_cost_to_blender(outWorkBook)
outWorkBook.close()

In [478]:
def to_excel_reliquat_demande(path):
    L=[[e.split('_')[0],e.split('_')[1],[Reliquat_Demand[e,t].x for t in periods]] for e in fertilizers ]
    S=[L[i][0:2]+L[i][2][:] for i in range(len(L)) ]
    df = pd.DataFrame (np.array(S),columns=['Province','Filière','Q2' , 'Q3' ,'Q4' ,'Q1'])
    ## save to xlsx file
    with pd.ExcelWriter(path,mode = 'a',engine ='openpyxl',if_sheet_exists='replace' ) as writer:

        df.to_excel(writer,index=False, sheet_name='reliquat_demande')
    return

In [479]:
def to_excel_Active_b(path):
    L=[[st,[Active_Blender[st,t].X for t in periods]] for st  in Storage_MP ]
    S=[L[i][0:1]+L[i][1][:] for i in range(len(L))]
    df = pd.DataFrame (np.array(S),columns=['Province','Q2' , 'Q3' ,'Q4' ,'Q1'])
    with pd.ExcelWriter(path,mode = 'a',engine ='openpyxl',if_sheet_exists='replace' ) as writer:
        df.to_excel(writer,index=False, sheet_name='Active_Blender')
    return

In [480]:
def to_excel_Active_Sb(path):
    L=[[st,[Active_SBlender[st,t].X for t in periods]] for st  in Storage_MP ]
    S=[L[i][0:1]+L[i][1][:] for i in range(len(L))]
    df = pd.DataFrame (np.array(S),columns=['Province','Q2' , 'Q3' ,'Q4' ,'Q1'])
    with pd.ExcelWriter(path,mode = 'a',engine ='openpyxl',if_sheet_exists='replace' ) as writer:
        df.to_excel(writer,index=False, sheet_name='Active_SBlender')
    return

In [481]:
def to_excel_Active_Downstream_Storage(path):
    L=[[st,[Active_Downstream_Storage[st,t].X for t in periods]] for st  in Storage_MP ]
    S=[L[i][0:1]+L[i][1][:] for i in range(len(L))]
    df = pd.DataFrame (np.array(S),columns=['Province','Q2' , 'Q3' ,'Q4' ,'Q1'])
    with pd.ExcelWriter(path,mode = 'a',engine ='openpyxl',if_sheet_exists='replace' ) as writer:
        df.to_excel(writer,index=False, sheet_name='Active_Downstream_Storage')
    return

In [482]:
def to_excel_Active_Upstream_Storage(path):
    L=[[st,[Active_Upstream_Storage[st,t].X for t in periods]] for st  in Storage_MP ]
    S=[L[i][0:1]+L[i][1][:] for i in range(len(L))]
    df = pd.DataFrame (np.array(S),columns=['Province','Q2' , 'Q3' ,'Q4' ,'Q1'])
    with pd.ExcelWriter(path,mode = 'a',engine ='openpyxl',if_sheet_exists='replace' ) as writer:
        df.to_excel(writer,index=False, sheet_name='Active_Upstream_Storage')
    return

In [483]:
def to_excel_supply_blender(path):
    columns = ['Supply','Blender','Matiere_premiere','Periode','Type of Blender','Quantité','freight_cost','cost_blender_total','average_cost_blender']
    Supply = []
    Matiere_premiere = []
    Blender = []
    Periode = []
    Type_of_Blender = []
    Quantité = []
    freight_cost = []
    cost_blender_total = []
    average_cost_blender = []
    for s in supply:
        for p in Province:
            for m in MP:
                if dict_Supply[s][m]>0:
                    for t in periods :
                        if Flow_Supply_Blender[(s,p,m,t)].X:
                            Supply.append(s)
                            Blender.append(p)
                            Matiere_premiere.append(m)
                            Periode.append(t)
                            Type_of_Blender.append("B")
                            Quantité.append(Flow_Supply_Blender[(s,p,m,t)].X)
                            freight_cost.append(dict_transAmont[s][p])
                            cost_blender_total.append((dict_MP[m]+dict_transAmont[s][p])*Flow_Supply_Blender[(s,p,m,t)].X)
                            average_cost_blender.append(dict_MP[m]+dict_transAmont[s][p])
                        if Flow_Supply_SBlender[(s,p,m,t)].X:
                            Supply.append(s)
                            Blender.append(p)
                            Matiere_premiere.append(m)
                            Periode.append(t)
                            Type_of_Blender.append("SB")
                            Quantité.append(Flow_Supply_SBlender[(s,p,m,t)].X)
                            freight_cost.append(dict_transAmont[s][p])
                            cost_blender_total.append((dict_MP[m]+dict_transAmont[s][p])*Flow_Supply_SBlender[(s,p,m,t)].X)
                            average_cost_blender.append(dict_MP[m]+dict_transAmont[s][p])


    # Create DataFrame from multiple lists
    df4 = pd.DataFrame(list(zip(Supply,Blender,Matiere_premiere,Periode,Type_of_Blender,Quantité,freight_cost,cost_blender_total,average_cost_blender)), columns=columns)    
    with pd.ExcelWriter(path,mode = 'a', engine = 'openpyxl',if_sheet_exists = 'replace') as writer:
        df4.to_excel(writer, sheet_name='flow_supply_blender', index=False)

In [484]:
def to_excel_stock_MP(path):
    columns =["Stockage_MP","Matiere premiere","Periode","Quantité","cost"]
    Stockage_MP = []
    Matiere_premiere = []
    Periode = []
    Quantité = []
    cost = []
    for st in Storage_MP :
        for mp in MP:
            for t in periods:
                if Stock_MP[(st,mp,t)].X >= 0.1:
                    Stockage_MP.append(st)
                    Matiere_premiere.append(mp)
                    Periode.append(t)
                    Quantité.append(Stock_MP[(st,mp,t)].X)
                    cost.append((Stock_MP[(st,mp,t)].X)*Donnees["Cout_de_stockage_MP"])

    # Create DataFrame from multiple lists
    df5 = pd.DataFrame(list(zip(Stockage_MP,Matiere_premiere,Periode,Quantité,cost)), columns=columns)  
    with pd.ExcelWriter(path,mode = 'a', engine = 'openpyxl',if_sheet_exists = 'replace') as writer:
        df5.to_excel(writer, sheet_name='stock_fin_periode_mp', index=False)

In [485]:
def to_excel_stock_engrais(path):
    columns =["Stockage_Engrais","Province","Culture","Periode","Quantité","cost"]
    Stockage_Engrais = []
    Province = []
    Culture = []
    Periode = []
    Quantité = []
    cost = []
    for sf in Storage_Fertilizer :
        for e in fertilizers:
            for t in periods:
                if Stock_Fertlizer[(sf,e,t)].X >= 0.1:
                    L = e.split("_")
                    Stockage_Engrais.append(sf)
                    Province.append(L[0])
                    Culture.append(L[1])
                    Periode.append(t)
                    Quantité.append(Stock_Fertlizer[(sf,e,t)].X)
                    cost.append((Stock_Fertlizer[(sf,e,t)].X)*Donnees["Cout_de_stockage_Engrais"])

    # Create DataFrame from multiple lists
    df6 = pd.DataFrame(list(zip(Stockage_Engrais,Province,Culture,Periode,Quantité,cost)), columns=columns)    
    with pd.ExcelWriter(path,mode = 'a', engine = 'openpyxl',if_sheet_exists = 'replace') as writer:
        df6.to_excel(writer, sheet_name='stock_fin_periode_engrais', index=False)

In [486]:
def to_excel_supply_stock(path):
    L = []
    k = Flow_Supply_StorageMP.keys()
    for i in range (len(k)):
        if  Flow_Supply_StorageMP[k[i]].X > 0:

            L.append([k[i][0], k[i][1], k[i][2], k[i][3], Flow_Supply_StorageMP[k[i]].X ,dict_transAmont[k[i][0]][k[i][1]],dict_MP[k[i][2]]*Flow_Supply_StorageMP[k[i]].X ,dict_MP[k[i][2]] ])


    df = pd.DataFrame(L, columns = ["Supply", "Stockage_mp", "Matiere premiere", "Periode", "Quantité","freight cost", "Cost_stock_total", "Average_cost_stock_mp"])
    with pd.ExcelWriter(path, mode="a", engine="openpyxl", if_sheet_exists='replace') as writer:
        df.to_excel(writer, sheet_name='Flow_supply_storage', index = False)

In [487]:
def to_excel_blender_stock(path):
    m = Flow_Blender_StorageFertlizer.keys()
    c=[]
    for i in range(len(m)):
        c.append(m[i][2].split("_"))

    k = []
    cost = 0
    for i in range(len(m)):
        if Flow_Blender_StorageFertlizer[m[i][0],m[i][1],m[i][2],m[i][3]].X > 0:
            cost = dict_transAval[m[i][0]][c[i][0]]
            k.append([m[i][0], m[i][1], c[i][0], c[i][1],m[i][3],Flow_Blender_StorageFertlizer[m[i][0],m[i][1],m[i][2],m[i][3]].X, cost])


    df2=pd.DataFrame(k, columns = ["Blender", "Stockage_Engrais", "Province", "Culture", "Periode", "Quantité", "freight_cost"])
    with pd.ExcelWriter(path, mode="a", engine="openpyxl", if_sheet_exists='replace') as writer:
        df2.to_excel(writer, sheet_name='Flow_blender_storage', index = False)

In [488]:
def to_excel_stock_province(path):
    d = Flow_StorageFertlizer_Province.keys()
    e=[]
    for i in range(len(d)):
        e.append(d[i][1].split("_"))

    j = []
    cost = 0
    for i in range(len(d)):
        if Flow_StorageFertlizer_Province[d[i][0], d[i][1], d[i][2]].X>0:
            cost = dict_transAval[d[i][0]][e[i][0]]
            j.append([d[i][0], e[i][0], e[i][1], d[i][2], Flow_StorageFertlizer_Province[d[i][0], d[i][1], d[i][2]].X, cost, cost, cost*Flow_StorageFertlizer_Province[d[i][0], d[i][1], d[i][2]].X])

    df3=pd.DataFrame(j, columns = ["Stockage_Engrais", "Province", "Culture", "Periode", "Quantité", "freight_cost", "average_cost_p", "Total_cost"])
    with pd.ExcelWriter(path, mode="a", engine="openpyxl", if_sheet_exists='replace') as writer:
        df3.to_excel(writer, sheet_name='flow_storage_to_province', index = False)

In [489]:
def Cost_to_provine(path):
    N = len(Flow_Blender_Province)
    listLignes = []
    for ligne in range(N):
        try :    
            province = Flow_Blender_Province.keys()[ligne][0]
            fertelizer = Flow_Blender_Province.keys()[ligne][1]
            filiere = fertelizer.split('_').pop()
            periode = Flow_Blender_Province.keys()[ligne][2]
            volumeBlender = Flow_Blender_Province[(province, fertelizer, periode)].X + Flow_SBlender_Province[(province, fertelizer, periode)].X
            volumeFertilizer = Flow_StorageFertlizer_Province[(province, fertelizer, periode)].X
            priceBlender = sum([ volumeBlender*dict_Demand[province][filiere][mp]*dict_MP[mp] for mp in MP ])
            priceFertilizer = sum([ volumeFertilizer*dict_Demand[province][filiere][mp]*dict_MP[mp] for mp in MP ])
            volumeTotal = volumeBlender + volumeFertilizer

            if volumeTotal != 0 :
                averageCost = ( priceBlender + priceFertilizer)/volumeTotal
                margeDist = averageCost * (1 + marge_distributeur)
                margeReve = margeDist * (1 + marge_revendeur)
                listLignes.append([province, filiere, periode, volumeBlender, priceBlender, volumeFertilizer, priceFertilizer, volumeTotal, averageCost, margeDist, margeReve])
        except :
            pass
    CostToProvince_Columns = ['province', 'filiere', 'periode', 'volumeBlender', 'priceBlender', 'volumeFertilizer', 'priceFertilizer', 'volumeTotal', 'averageCost', 'marge_distributeur', 'marge_revendeur']
    dataframe_CostToProvince = pd.DataFrame(listLignes, columns = CostToProvince_Columns)
    filename = path
    sheetname = "Cost to province"
    with pd.ExcelWriter(filename, mode="a", engine="openpyxl", if_sheet_exists='replace') as writer:
        dataframe_CostToProvince.to_excel(writer, sheet_name= sheetname, index=False)  


In [490]:
def Cost_to_supply(path):
    supply_Excel = [ s for s in supply for mp in MP for p in periods]
    MP_Excel = [ mp for s in supply for mp in MP for p in periods]
    periods_Excel = [ p for s in supply for mp in MP for p in periods]
    provinces = Storage_MP
    supply_Column = []
    MP_Column = []
    periods_Column = []
    volume_Column = []
    cost_Column = []
    unitCost_Column = []

    for raw in range(len(supply_Excel)):
        volume_supply = 0
        s = supply_Excel[raw]
        mp = MP_Excel[raw]
        p = periods_Excel[raw]
        for province in provinces :
            toStorageMP = Flow_Supply_StorageMP[(s, province, mp, p)].X
            toBlender = Flow_Supply_Blender[(s, province, mp, p)].X
            toSBlender = Flow_Supply_SBlender[(s, province, mp, p)].X
            volume_supply += toStorageMP + toBlender + toSBlender
        cost_supply = dict_MP[mp]
        if volume_supply != 0 :
            supply_Column.append(supply_Excel[raw])
            MP_Column.append(MP_Excel[raw])
            periods_Column.append(periods_Excel[raw])
            volume_Column.append(volume_supply)
            cost_Column.append(cost_supply*volume_supply)
            unitCost_Column.append(cost_supply)
    cost_to_supply = pd.DataFrame({'Supply': supply_Column,
                          'MP': MP_Column,
                          'periode': periods_Column,
                          'volume_s': volume_Column,
                          'cost_s': cost_Column,
                          'average_cost_supply': unitCost_Column })
    filename = path
    sheetname = "Cost to supply"
    with pd.ExcelWriter(filename, mode="a", engine="openpyxl", if_sheet_exists='replace') as writer:
        cost_to_supply.to_excel(writer, sheet_name= sheetname, index=False)  

In [459]:
Cost_to_supply(pathr)
to_excel_supply_blender(pathr)
to_excel_supply_stock(pathr)
to_excel_reliquat_demande(pathr)
to_excel_Active_b(pathr)
to_excel_Active_Sb(pathr)
to_excel_Active_Downstream_Storage(pathr)
to_excel_Active_Upstream_Storage(pathr)
to_excel_stock_MP(pathr)
to_excel_stock_engrais(pathr)
Cost_to_provine(pathr)
to_excel_blender_stock(pathr)
to_excel_stock_province(pathr)